In [10]:
import csv
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM, Dropout,Embedding, Bidirectional
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
vocab_size = 5000
embedding_dim = 64
max_length = 200
oov_tok = '<OOV>' #  Out of Vocabulary
training_portion = 0.8

In [12]:
articles = []
labels = []

with open("Data/arabic_main.csv", 'r',encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[1])
        article = row[0]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            article = article.replace(token, ' ')
            article = article.replace(' ', ' ')
        articles.append(article)

In [13]:
len(labels),len(articles)

(8752, 8752)

In [14]:
print(labels[0])

weather


In [15]:
train_size = int(len(articles) * training_portion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

In [16]:
print('train_articles' ,len(train_articles))
print('train_labels', len(train_labels))
print('validation_articles', len(validation_articles))
print('validation_labels', len(validation_labels))

train_articles 7001
train_labels 7001
validation_articles 1751
validation_labels 1751


In [17]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index

In [18]:
import pickle
filename = f'../../utils/tokenizer.sav'
pickle.dump(tokenizer, open(filename, 'wb'))

In [19]:
word_index

{'<OOV>': 1,
 'هل': 2,
 '،': 3,
 'ما': 4,
 'من': 5,
 'الطقس': 6,
 'يوم': 7,
 'الساعة': 8,
 'الخاص': 9,
 'يمكنك': 10,
 'اليوم': 11,
 'سيكون': 12,
 'التقويم': 13,
 'موعد': 14,
 'كيف': 15,
 'مساءً': 16,
 'اليوم؟': 17,
 'الأسبوع': 18,
 'الأول': 19,
 'يمكن': 20,
 'المركز': 21,
 'التحقق': 22,
 'تمام': 23,
 'حسنًا': 24,
 'تخبرني': 25,
 'أريد': 26,
 'فريق': 27,
 'وقت': 28,
 'ستكون': 29,
 'أعرف': 30,
 'لعبوا': 31,
 'ماذا': 32,
 'فضلك': 33,
 'حارس': 34,
 'الحدث': 35,
 'لدي': 36,
 '30': 37,
 'توقعات': 38,
 '15': 39,
 'بي': 40,
 'مكان': 41,
 'أود': 42,
 'يوليو؟': 43,
 'كم': 44,
 'الجو': 45,
 'أحتاج': 46,
 'نعم': 47,
 'في': 48,
 'السبت': 49,
 'يلعبون': 50,
 'متى': 51,
 'مارس': 52,
 'إضافة': 53,
 'المقبل؟': 54,
 'سيلعبون': 55,
 'قل': 56,
 'لعب': 57,
 'مرحبًا': 58,
 'عدد': 59,
 'يحتل': 60,
 'صباحًا': 61,
 'ومن': 62,
 'طبيب': 63,
 'شكرًا': 64,
 'شكرا': 65,
 'يلعب': 66,
 'حدد': 67,
 'موعدًا': 68,
 '3': 69,
 'أتساءل': 70,
 'آخر': 71,
 'الأحداث': 72,
 '4': 73,
 'مارس؟': 74,
 'ليوم': 75,
 'الشهر': 76,
 'ا

In [20]:
tokenizer.texts_to_sequences(['what will be the weather tommorrow morning ?'])

[[1, 1, 1, 680, 1, 1, 1]]

In [21]:
train_sequences = tokenizer.texts_to_sequences(train_articles)
len(train_sequences)

7001

In [22]:
train_sequences[0]

[2, 12, 45, 933, 684, 448, 7, 1449]

In [23]:
train_padded = pad_sequences(train_sequences, maxlen=max_length,)

In [24]:
len(train_padded),len(train_padded[0])

(7001, 200)

In [25]:
validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length)

In [26]:
set(labels)

{'general',
 'greeting',
 'recommendation',
 'schedule',
 'sports',
 'thank',
 'weather'}

In [27]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

In [28]:
validation_label_seq

array([[3],
       [1],
       [4],
       ...,
       [5],
       [4],
       [1]])

In [29]:
# train_labels[744], training_label_seq[744]

In [30]:
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim ))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(embedding_dim)))
model.add(Dense(8,activation='softmax' ))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          320000    
                                                                 
 dropout (Dropout)           (None, None, 64)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dense (Dense)               (None, 8)                 1032      
                                                                 
Total params: 387,080
Trainable params: 387,080
Non-trainable params: 0
_________________________________________________________________


In [31]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

C:\Users\dell\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [32]:
num_epochs = 10
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), verbose=2)

Epoch 1/10
219/219 - 39s - loss: 1.2146 - accuracy: 0.5798 - val_loss: 0.5962 - val_accuracy: 0.8658 - 39s/epoch - 179ms/step
Epoch 2/10
219/219 - 42s - loss: 0.3883 - accuracy: 0.8996 - val_loss: 0.3054 - val_accuracy: 0.9109 - 42s/epoch - 193ms/step
Epoch 3/10
219/219 - 39s - loss: 0.1928 - accuracy: 0.9476 - val_loss: 0.2112 - val_accuracy: 0.9389 - 39s/epoch - 179ms/step
Epoch 4/10
219/219 - 36s - loss: 0.1135 - accuracy: 0.9723 - val_loss: 0.1944 - val_accuracy: 0.9509 - 36s/epoch - 162ms/step
Epoch 5/10
219/219 - 38s - loss: 0.0772 - accuracy: 0.9793 - val_loss: 0.1805 - val_accuracy: 0.9520 - 38s/epoch - 172ms/step
Epoch 6/10
219/219 - 38s - loss: 0.0585 - accuracy: 0.9841 - val_loss: 0.1781 - val_accuracy: 0.9532 - 38s/epoch - 175ms/step
Epoch 7/10
219/219 - 42s - loss: 0.0476 - accuracy: 0.9866 - val_loss: 0.2019 - val_accuracy: 0.9480 - 42s/epoch - 193ms/step
Epoch 8/10
219/219 - 40s - loss: 0.0370 - accuracy: 0.9901 - val_loss: 0.2037 - val_accuracy: 0.9497 - 40s/epoch - 184

In [41]:
txt_list = ["ما حال الطقس اليوم ؟","احجز لى معاد اليوم الساعة 5","كان بوم جميل جدا"," اقترح فيلم جديد لاشاهده"]
labels = ['sports','schedule','general','weather','recommendation','greeting','thank']
answer = list()
for txt in txt_list:
    seq = tokenizer.texts_to_sequences([txt])
    padded = pad_sequences(seq, maxlen=max_length)
    pred = model.predict(padded)
    try:
        label = labels[np.argmax(pred)-1] 
        answer.append(label)
    except:
        answer.append('general')
print(answer)

1/1 [==============================] - 0s 104ms/step
['weather', 'schedule', 'general', 'recommendation']


In [38]:
model.save("models")

INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets


In [39]:
from tensorflow import keras

m = keras.models.load_model("models")

In [42]:
answer = list()
for txt in txt_list:
    seq = tokenizer.texts_to_sequences([txt])
    padded = pad_sequences(seq, maxlen=max_length)
    pred = m.predict(padded)
    try:
        label = labels[np.argmax(pred)-1] 
        answer.append(label)
    except:
        answer.append('general')
print(answer)

1/1 [==============================] - 0s 55ms/step
['weather', 'schedule', 'general', 'recommendation']
